In [34]:
import preprocessing
from sklearn.model_selection import KFold
import argparse
from model import *
from train import *
import torch.optim as optim


In [9]:
subjects_adj_before,subjects_labels_before = preprocessing.load_data(25629,25830)

/vol/biomedic3/bglocker/ugproj2324/nns20/y4/DGL/super-brains/GSR-Net/preprocessing.py:41: RuntimeWarning: Mean of empty slice
  col_mean = np.nanmean(roi, axis=0)


In [6]:
subjects_adj, subjects_labels, test_adj, test_labels = preprocessing.data()

/vol/biomedic3/bglocker/ugproj2324/nns20/y4/DGL/super-brains/GSR-Net/preprocessing.py:41: RuntimeWarning: Mean of empty slice
  col_mean = np.nanmean(roi, axis=0)


In [15]:
# compare the adjacency matrix before and after preprocessing
if subjects_adj_before.all() == subjects_adj.all():
    print("The adjacency matrix before and after preprocessing is the same?")


The adjacency matrix before and after preprocessing is the same?


In [21]:
num_splt = 5
epochs = 200
lr = 0.0001
lmbda = 16
lr_dim = 160
hr_dim = 320
hidden_dim = 320
padding = 26


In [27]:
cv = KFold(n_splits=5, random_state=42, shuffle=True)

In [37]:
args = argparse.Namespace()
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.lr = lr
args.epochs = epochs


In [32]:
# ks = [0]
ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args)

In [38]:
print(model)
optimizer = optim.Adam(model.parameters(), lr=args.lr)

for train_index, test_index in cv.split(subjects_adj):
    subjects_adj, test_adj, subjects_ground_truth, test_ground_truth = subjects_adj[
        train_index], subjects_adj[test_index], subjects_labels[train_index], subjects_labels[test_index]
    train(model, optimizer, subjects_adj, subjects_ground_truth, args)
    test(model, test_adj, test_ground_truth, args)

GSRNet(
  (layer): GSRLayer()
  (net): GraphUnet(
    (start_gcn): GCN(
      (proj): Linear(in_features=160, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (bottom_gcn): GCN(
      (proj): Linear(in_features=320, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
    (end_gcn): GCN(
      (proj): Linear(in_features=640, out_features=320, bias=True)
      (drop): Dropout(p=0, inplace=False)
    )
  )
  (gc1): GraphConvolution()
  (gc2): GraphConvolution()
)


RuntimeError: This function was deprecated since version 1.9 and is now removed. The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.

L, _ = torch.symeig(A, upper=upper) should be replaced with:
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')

and

L, V = torch.symeig(A, eigenvectors=True) should be replaced with:
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L')